In [1]:
import os
import numpy as np
import pandas as pd

import xml.etree.ElementTree as ET
import cv2

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, BatchNormalization, Dropout, Flatten, Dense, Input, Concatenate
from tensorflow.keras import Model

from sklearn.model_selection import train_test_split

from tensorflow.keras.utils import to_categorical

import tensorflow as tf

from tensorflow.keras.callbacks import Callback, EarlyStopping,ModelCheckpoint, ReduceLROnPlateau

!wget https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
from helper_functions import create_tensorboard_callback, plot_loss_curves, unzip_data, compare_historys, walk_through_dir, pred_and_plot

2024-04-10 01:40:42.663432: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-10 01:40:42.663540: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-10 01:40:42.803497: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


--2024-04-10 01:40:53--  https://raw.githubusercontent.com/mrdbourke/tensorflow-deep-learning/main/extras/helper_functions.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.111.133, 185.199.109.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10246 (10K) [text/plain]
Saving to: 'helper_functions.py'

helper_functions.py 100%[===================>]  10.01K  --.-KB/s    in 0s      

2024-04-10 01:40:54 (61.4 MB/s) - 'helper_functions.py' saved [10246/10246]



In [2]:
store_names = []
directory = "/kaggle/input/logodet3k/LogoDet-3K/Food"
for item in os.listdir(directory):
    item_path = os.path.join(directory, item)
    if os.path.isdir(item_path):
        store_names.append(item)

In [3]:
def create_model(input_image_shape, input_xml_shape, num_classes):
    # Pretrained model
    
    pretrained_model = tf.keras.applications.EfficientNetB0(
        input_shape=(128, 128, 3),
        include_top=False,
        weights='imagenet',
        pooling='max'
    )

    # Image input branch
    image_input = Input(shape=input_image_shape)
    image_branch = pretrained_model(image_input, training=False)  # Freeze pretrained layers

    # XML input branch
    xml_input = Input(shape=input_xml_shape)
    xml_branch = Dense(128, activation='relu')(xml_input)

    # Combine image and XML branches
    combined = Concatenate()([image_branch, xml_branch])

    # Output layer
    output = Dense(num_classes, activation='softmax')(combined)

    # Define the model
    model = Model(inputs=[image_input, xml_input], outputs=output)
    return model

In [4]:
def extract_label_and_bbox_from_xml(xml_path):
    tree = ET.parse(xml_path)
    root = tree.getroot()
    
    
    # Extract label
    label = root.find('object/name').text
    
    # Extract bounding box coordinates
    xmin = int(root.find('object/bndbox/xmin').text)
    ymin = int(root.find('object/bndbox/ymin').text)
    xmax = int(root.find('object/bndbox/xmax').text)
    ymax = int(root.find('object/bndbox/ymax').text)
    bbox = [xmin, ymin, xmax, ymax]
    
    return label, bbox

In [5]:
data = []

total_files = sum(len(files) for _, _, files in os.walk(directory)) // 2
files_processed = 0

for label, store_name in enumerate(store_names):
    store_path = os.path.join(directory, store_name)  
    for file in os.listdir(store_path):
        # Filter out non-image files
        if file.lower().endswith(('.jpg', '.jpeg', '.png')):
            img_path = os.path.join(store_path, file)
            img = cv2.imread(img_path, cv2.IMREAD_COLOR)
            img = cv2.resize(img, (128, 128))
            # Extract label and bounding box from corresponding XML file
            xml_file = file[:-4] + '.xml'  # Assuming XML file has the same name as the image file
            xml_path = os.path.join(store_path, xml_file)
            _, bbox = extract_label_and_bbox_from_xml(xml_path)
            data.append([img, label, bbox])
            
             # Increment the count of processed files
            files_processed += 1
            # Print progress every 10% of completion
            if files_processed % (total_files // 10) == 0:
                print(f"Progress: {files_processed}/{total_files} files processed ({files_processed / total_files * 100:.0f}%)")

Progress: 10670/53350 files processed (20%)
Progress: 16005/53350 files processed (30%)
Progress: 21340/53350 files processed (40%)
Progress: 26675/53350 files processed (50%)
Progress: 32010/53350 files processed (60%)
Progress: 37345/53350 files processed (70%)
Progress: 42680/53350 files processed (80%)
Progress: 48015/53350 files processed (90%)
Progress: 53350/53350 files processed (100%)


In [6]:
X = []
Y = []
XML = []
for features,label,xml in data:
    X.append([features, xml])
    Y.append(label)
xtrainD,xtestD,ytrain,ytest = train_test_split(X,Y,test_size=0.2,random_state=56)
xtrain = np.array([a[0] for a in xtrainD])
xtest = np.array([a[0] for a in xtestD])
xtrain_xml = np.array([a[1] for a in xtrainD])
xtest_xml = np.array([a[1] for a in xtestD])

ytrain = np.array(ytrain)
ytest = np.array(ytest)

ytrain = to_categorical(ytrain,len(store_names))
ytest = to_categorical(ytest,len(store_names))

In [7]:
# Create checkpoint callback
checkpoint_path = "classification_model_checkpoint.weights.h5"
checkpoint_callback = ModelCheckpoint(checkpoint_path,
                                      save_weights_only=True,
                                      monitor="val_accuracy",
                                      save_best_only=True)

# Setup EarlyStopping callback to stop training if model's val_loss doesn't improve for 3 epochs
early_stopping = EarlyStopping(monitor = "val_loss", # watch the val loss metric
                               patience = 7,
                               restore_best_weights = True) # if val loss decreases for 3 epochs in a row, stop training

reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=3, min_lr=1e-6)

In [8]:
input_image_shape = xtrain.shape[1:]  # Shape of input images
input_xml_shape = (4,)  # Shape of input XML data (adjust according to your data)
num_classes = len(store_names)  # Number of output classes
nn = create_model(input_image_shape, input_xml_shape, num_classes)

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


In [9]:
nn.compile(loss='categorical_crossentropy',optimizer='adam',metrics=["accuracy"])
final_model = nn.fit([xtrain, xtrain_xml],
                     ytrain,
                     validation_data=([xtest, xtest_xml], ytest),
                     epochs=100,
                    callbacks=[
                        early_stopping,
                        create_tensorboard_callback("training_logs", 
                                    "classification"),
                        checkpoint_callback,
                        reduce_lr
    ])

Saving TensorBoard log files to: training_logs/classification/20240410-015424
Epoch 1/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 215s 99ms/step - accuracy: 0.1609 - loss: 8.4859 - val_accuracy: 0.5348 - val_loss: 2.3821 - learning_rate: 0.0010
Epoch 2/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 70s 52ms/step - accuracy: 0.6235 - loss: 1.7704 - val_accuracy: 0.6602 - val_loss: 1.7256 - learning_rate: 0.0010
Epoch 3/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 70s 52ms/step - accuracy: 0.7656 - loss: 0.9841 - val_accuracy: 0.6978 - val_loss: 1.5386 - learning_rate: 0.0010
Epoch 4/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 70s 52ms/step - accuracy: 0.8275 - loss: 0.6698 - val_accuracy: 0.7078 - val_loss: 1.5381 - learning_rate: 0.0010
Epoch 5/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 70s 52ms/step - accuracy: 0.8683 - loss: 0.4918 - val_accuracy: 0.7351 - val_loss: 1.5194 - learning_rate: 0.0010
Epoch 6/100
1334/1334 ━━━━━━━━━━━━━━━━━━━━ 69s 52ms/step - accuracy: 0.8898 - loss: 0.3895 - val_accuracy: 0.7088 - val_loss: 1.6930 - l

Saving the model

In [10]:
nn.save("nn128.keras")

In [11]:
nn.save_weights("nn128.weights.h5")

Testing the model

In [12]:
op = cv2.imread('/kaggle/input/logodet3k/LogoDet-3K/Food/7-Up/12.jpg')
op = cv2.resize(op,(128,128))
op = np.array(op)
#op = op.reshape(1,30,30,3)



x_min, y_min = 5, 10
x_max, y_max = 20, 25
op_xml = np.array([x_min, x_max, y_min, y_max])
op_image = np.array(op).reshape(1, *op.shape)
op_xml = np.array(op_xml).reshape(1, *op_xml.shape)

model = nn.predict([op_image, op_xml])

# Convert to DataFrame
val = []
for i in model:
    val.append(i)
data_fram = pd.DataFrame([store_names,val[0]]).T
data_fram.columns = ["Outlate","Output"]

# Final Output
op_final = data_fram.groupby('Output').max().tail(1).values[0][0]

predicted_class_index = np.argmax(model)

    # Get the predicted class associated probability
probability = model.squeeze()[predicted_class_index] * 100  # Convert to percentage
print(f"Given Image is {op_final} Store with a probability of {probability:.2f}%")

1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step
Given Image is 7-Up Store with a probability of 99.95%
